<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/sk_gen_ai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/sk_gen_ai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
===================================================
Author: Sheldon Kemper
Role: Data Engineering Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/sheldon-kemper
Date: 2025-02-04
Version: 1.1

Description:
    This notebook contains a class-based implementation of a Retrieval Augmented Generation (RAG) engine
    designed to analyze bank quarterly earnings call transcripts (in PDF format) stored on Google Drive.
    The code performs the following tasks:

    1. Configures an LLM pipeline using a SmolLM-based model for text generation.
    2. Sets up sentence-transformer based embeddings for document vectorization.
    3. Loads and splits PDF documents from a specified directory.
    4. Chunks the documents and builds a vector index using Chroma.
    5. Retrieves context relevant to user queries from the vector index.
    6. Implements a fallback mechanism for queries unrelated to the provided data.
    7. Maintains conversation memory for interactive sessions.
    8. Supports both interactive and programmatic prompt-based querying.
===================================================
"""

'\n===================================================\nAuthor: Sheldon Kemper\nRole: Data Engineering Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/sheldon-kemper\nDate: 2025-02-04\nVersion: 1.1\n\nDescription:\n    This notebook contains a class-based implementation of a Retrieval Augmented Generation (RAG) engine\n    designed to analyze bank quarterly earnings call transcripts (in PDF format) stored on Google Drive.\n    The code performs the following tasks:\n\n    1. Configures an LLM pipeline using a SmolLM-based model for text generation.\n    2. Sets up sentence-transformer based embeddings for document vectorization.\n    3. Loads and splits PDF documents from a specified directory.\n    4. Chunks the documents and builds a vector index using Chroma.\n    5. Retrieves context relevant to user queries from the vector index.\n    6. Implements a fallback mechanism for queries unrelated to the provided data.\n    7. Maintains co

In [6]:
# install langchain-community
!pip install -q langchain-community pypdf tiktoken chromadb sentence-transformers > /dev/null 2>&1

In [3]:
from google.colab import drive
# Mount Google Drive to the root location with force_remount
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:

import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain

class BankEarningsChatbot:
    """
    A class-based implementation of an LLM Retrieval Augmented Generation (RAG) engine
    designed to analyze bank quarterly earnings call transcripts. It loads PDF documents,
    builds a Chroma vector index, and sets up an interactive conversational chain for prompt-based queries.
    """

    def __init__(self, pdf_folder, persist_directory="test_index", max_length=256):
        self.pdf_folder = pdf_folder
        self.persist_directory = persist_directory

        # Set up the LLM pipeline using the Flan-T5 model.
        self._setup_llm(max_length)

        # Configure embeddings.
        self._setup_embeddings()

        # Load documents from PDF files.
        self._load_documents()

        # Build the vector index.
        self._build_vector_index()

        # Configure retriever from the persisted vector database.
        self._setup_retriever()

        # Initialize conversation memory.
        self.memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)

        # Set up the conversational retrieval chain.
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.retriever,
            memory=self.memory,
            verbose=False
        )

    def _setup_llm(self, max_length):
        """
        Configures the language model pipeline using a T5 model.
        """
        self.model_name = "google/flan-t5-large"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name, device_map="auto", torch_dtype=torch.float16)
        # Using the text2text-generation pipeline for T5
        self.pipe = pipeline(
            "text2text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=max_length,
            temperature=0.5,
            top_p=0.8,
            do_sample=True
        )
        self.llm = HuggingFacePipeline(pipeline=self.pipe)

    def _setup_embeddings(self):
        """
        Initializes the sentence-transformer based embeddings.
        """
        self.embedding_model = "sentence-transformers/all-mpnet-base-v2"
        self.embeddings = HuggingFaceEmbeddings(model_name=self.embedding_model)

    def _load_documents(self):
        """
        Loads and splits PDF documents from the specified folder.
        """
        self.documents = []
        file_names = [f for f in os.listdir(self.pdf_folder) if f.endswith(".pdf")]
        for pdf_file in file_names:
            pdf_path = os.path.join(self.pdf_folder, pdf_file)
            try:
                loader = PyPDFLoader(pdf_path, extract_images=False)
                self.documents.extend(loader.load_and_split())
                print(f"Loaded: {pdf_file}")
            except Exception as e:
                print(f"Error loading {pdf_file}: {e}")

    def _build_vector_index(self):
        """
        Chunks documents and builds a Chroma vector index.
        """
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        chunks = text_splitter.split_documents(self.documents)
        # Remove duplicate chunks.
        chunks = self.remove_duplicate_chunks(chunks)
        self.chunks = chunks
        self.db = Chroma.from_documents(chunks, embedding=self.embeddings, persist_directory=self.persist_directory)
        self.db.persist()

    def _setup_retriever(self):
        """
        Initializes the retriever from the persisted vector database.
        """
        self.vectordb = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)
        self.retriever = self.vectordb.as_retriever(search_kwargs={"k": 3})

    @staticmethod
    def remove_duplicate_chunks(chunks):
        """
        Eliminates duplicate document chunks based on their content.
        """
        seen = set()
        unique_chunks = []
        for chunk in chunks:
            chunk_text = chunk.page_content.strip()
            if chunk_text not in seen:
                seen.add(chunk_text)
                unique_chunks.append(chunk)
        return unique_chunks

    def truncate_context(self, context_list, max_tokens=1000):
        """
        Truncates the retrieved context to avoid overloading the model's input.
        """
        truncated_docs = []
        current_tokens = 0
        for doc in context_list:
            doc_tokens = len(self.tokenizer.encode(doc.page_content))
            if current_tokens + doc_tokens <= max_tokens:
                truncated_docs.append(doc)
                current_tokens += doc_tokens
            else:
                break
        return truncated_docs

    @staticmethod
    def clean_user_input(user_input):
        """
        Cleans and standardizes user input.
        """
        return user_input.strip().replace("\n", " ").replace("\t", " ")

    def reset_memory_if_needed(self):
        """
        Clears conversation history if the number of exchanges exceeds a threshold.
        """
        if len(self.memory.chat_memory.messages) > 6:
            print("\nMemory Full: Resetting Conversation History...\n")
            self.memory.clear()

    def format_response(self, question, response):
        """
        Formats the output to clearly present both the question and the answer.
        """
        response_text = response.strip()
        unwanted_phrases = [
            "Use the following pieces of context",
            "If you don't know the answer, just say that you don't know",
            "Don't try to make up an answer."
        ]
        for phrase in unwanted_phrases:
            if phrase in response_text:
                response_text = response_text.split(phrase)[-1].strip()
        return f"Question: {question}\nHelpful Answer: {response_text}"

    def trim_final_input(self, question, context, max_tokens=1024):
        """
        Truncates the final input to meet the token limit, preserving document metadata.
        """
        system_message = (
            "You are analyzing a bank's quarterly earnings call transcript.\n"
            "Extract and summarize key financial insights, avoiding unnecessary details.\n"
            "If the answer isn't found, respond with 'I don't know.'\n"
            "Provide sources for your answers at the end."
        )
        input_text = f"{system_message}\n\nContext:\n{context}\n\nQuestion: {question}"
        tokens = self.tokenizer.encode(input_text, truncation=True, max_length=max_tokens)
        return self.tokenizer.decode(tokens)

    def answer_question(self, question):
        """
        Processes the user query: retrieves context, prepares the prompt,
        and returns a formatted answer. If no relevant documents are retrieved,
        a fallback message is returned.
        """
        question = self.clean_user_input(question)
        self.reset_memory_if_needed()

        # Retrieve and process context.
        context = self.retriever.get_relevant_documents(question)
        context = self.remove_duplicate_chunks(context)
        context = self.truncate_context(context, max_tokens=800)

        # Fallback: if no relevant context is found.
        if not context:
            return f"Question: {question}\nHelpful Answer: I don't have information regarding that query."

        print("\nRetrieved Context:")
        for doc in context:
            source = doc.metadata.get('source', 'Unknown Source')
            page = doc.metadata.get('page', 'Unknown Page')
            print(f"- Source: {source}, Page: {page}")

        formatted_input = self.trim_final_input(question, context, max_tokens=1024)
        response = self.qa_chain({"question": formatted_input})
        return self.format_response(question, response['answer'])

    def run_chatbot(self):
        """
        Initiates an interactive loop for prompt-based queries.
        """
        print("\n💬 Bank Earnings Chatbot (Type 'exit' to stop)")
        while True:
            user_input = input("\nYou: ")
            if user_input.lower() == "exit":
                print("\nExiting Chatbot. Have a great day!")
                break
            answer = self.answer_question(user_input)
            print("\n" + answer)

Based on the transcripts and retrieval‐augmented setup, here are some recommendations for crafting prompts that are likely to yield the most accurate and domain‐specific responses:

- **Be Specific About the Timeframe:**  
  Instead of asking “What were the key insights?” specify the quarter or transcript you’re interested in. For example:  
  - "What were the key financial insights from the Q4 2023 earnings call?"  
  - "Summarize the main drivers of revenue in the Q1 2023 transcript."

- **Target Specific Financial Metrics or Themes:**  
  Focus on particular areas the transcripts cover, such as revenue trends, expense drivers, or capital performance. For example:  
  - "How did revenue change compared to the previous quarter in the Q4 2023 earnings call?"  
  - "What were the primary expense drivers discussed in the Q4 2023 transcript?"

- **Incorporate Domain-Specific Language:**  
  Use terminology that reflects the financial domain to guide the model. For example:  
  - "What risk factors and forward-looking statements were highlighted in the Q3 2023 transcript?"  
  - "Outline the key operational challenges and strategic responses mentioned in the earnings call."

- **Prompt for Summaries and Insights:**  
  Asking for summaries can help the model focus on extracting concise information from large volumes of text. For example:  
  - "Provide a concise summary of the key financial insights from the Q4 2023 earnings transcript, including revenue, expenses, and capital allocation."  
  - "What are the overall sentiments and key management strategies discussed in the transcript?"

By tailoring your queries with specific quarters, financial metrics, and industry language, you guide the retrieval and summarization process more effectively. This structured approach should lead to more precise and contextually relevant responses from your system.

In [ ]:
# ----------------------------
# Example usage of the BankEarningsChatbot class with T5
# ----------------------------

# Define your PDF folder path (ensure this path contains your earnings transcripts in PDF format).
pdf_folder = "/content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan"

# Instantiate the chatbot object.
chatbot = BankEarningsChatbot(pdf_folder)

# Option 1: Launch an interactive session.
# chatbot.run_chatbot()

# Option 2: Process a list of prompts programmatically.
prompts = [
    "What were the key insights from the latest earnings call?",
    "How did revenue change compared to the previous quarter?",
    "What risk factors were identified in the transcript?",
    "What is the overall sentiment of the earnings call?"  # This might trigger the fallback if off-topic.
]

for prompt in prompts:
    response = chatbot.answer_question(prompt)
    print("Question:", prompt)
    print("Response:", response)
    print("-" * 60)

Device set to use cpu


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loaded: 1q23-earnings-transcript.pdf
Loaded: 2q23-earnings-transcript.pdf
Loaded: 4q24-earnings-transcript.pdf
Loaded: jpm-1q24-earnings-call-transcript.pdf
Loaded: jpm-2q24-earnings-call-transcript-final.pdf
Loaded: jpm-3q23-earnings-call-transcript.pdf
Loaded: jpm-4q23-earnings-call-transcript.pdf
Loaded: jpmc-third-quarter-2024-earnings-conference-call-transcript.pdf


<ipython-input-7-4d896d1220d7>:100: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  self.db.persist()
<ipython-input-7-4d896d1220d7>:106: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectordb = Chroma(persist_directory=self.persist_directory, embedding_function=self.embeddings)
<ipython-input-7-4d896d1220d7>:39: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferWindowMemory(k=3, memory_key="chat_history", return_messages=True)
<ipython-input-7-4d896d1220d7>:192: LangChainDeprecationWarning: The


Retrieved Context:
- Source: /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan/jpm-2q24-earnings-call-transcript-final.pdf, Page: 2
- Source: /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan/4q24-earnings-transcript.pdf, Page: 0
- Source: /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan/1q23-earnings-transcript.pdf, Page: 3
